In [0]:
import warnings 
warnings.filterwarnings('ignore')

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Flatten 
from keras.layers import Dense
from keras.layers import Dropout, LocallyConnected2D, BatchNormalization, Activation, SeparableConv2D
from keras import layers
from keras.models import Model
from keras.regularizers import l2  
from keras.layers import Input

Using TensorFlow backend.


In [3]:
from google.colab  import drive
drive.mount('/content/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = '/content/gdrive/My Drive/THI'

In [5]:
import pandas as pd
emotion = pd.read_csv(path + '/emotion.csv')
emotion.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [0]:
import numpy as np
def reshape_string_listofnumber(df, index):
  string = df[index]
  string = string.split(' ')
  lst = np.array([int(charac) for charac in string])

  return lst

def reshape_dataset_matrix(df):
  matrix = np.empty((df.shape[0], 48, 48, 1))
  for i in range(df.shape[0]):
    ele = reshape_string_listofnumber(df, i)
    ele = np.reshape(ele,(48,48,1))
    matrix[i] = ele

  return matrix

In [0]:
X_train = emotion.pixels[emotion.Usage == 'Training']
X_test = emotion.pixels[emotion.Usage == 'PrivateTest'].reset_index(drop = True)

X_train = reshape_dataset_matrix(X_train)
X_test = reshape_dataset_matrix(X_test)

In [0]:
from keras.utils import to_categorical
Y_train = emotion.emotion[emotion.Usage == 'Training']
Y_test = emotion.emotion[emotion.Usage == 'PrivateTest'].reset_index(drop = True)

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [0]:
def emotions(emo):
  if (emo) == 0:
    return 'giận dữ'
  elif (emo) == 1:
    return 'ghê tởm'
  elif (emo) == 2:
    return 'sợ hãi'
  elif (emo) == 3:
    return 'vui vẻ'
  elif (emo) == 4:
    return 'buồn'
  elif (emo) == 5:
    return 'bất ngờ'
  else:
    return 'trung lập'

In [0]:
from keras import applications 

model = applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (224,224,3))

The dataset is large and not similar to original dataset

In [0]:
x = model.output

regularization = l2(0.01)

    # base
img_input = Input((48,48,1))
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,
                                            use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

    # module 1
residual = Conv2D(16, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

    # module 2
residual = Conv2D(32, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

    # module 3
residual = Conv2D(64, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

    # module 4
residual = Conv2D(128, (1, 1), strides=(2, 2),
                      padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same',
                        kernel_regularizer=regularization,
                        use_bias=False)(x)
x = BatchNormalization()(x)

x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

x = Conv2D(7, (3, 3),
            #kernel_regularizer=regularization,
            padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax',name='predictions')(x)

model = Model(img_input, output)

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 46, 46, 8)    72          input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 46, 46, 8)    32          conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_154 (Activation)     (None, 46, 46, 8)    0           batch_normalization_15[0][0]     
__________________________________________________________________________________________

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 5), ModelCheckpoint(path + '/emotion_off_cau3.hdf5', 'val_loss', save_best_only = True)]

In [0]:
from keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)
model.fit_generator(data_generator.flow(X_train, Y_train, 32),
                    steps_per_epoch=897,
                    epochs=80, verbose=1, callbacks=callbacks,
                    validation_data=(X_test,Y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/80
897/897 [==============================] - 51s 57ms/step - loss: 1.7702 - acc: 0.3296 - val_loss: 1.6756 - val_acc: 0.3943
Epoch 2/80
897/897 [==============================] - 45s 50ms/step - loss: 1.5146 - acc: 0.4304 - val_loss: 1.4154 - val_acc: 0.4698
Epoch 3/80
897/897 [==============================] - 45s 50ms/step - loss: 1.3965 - acc: 0.4768 - val_loss: 1.4390 - val_acc: 0.4720
Epoch 4/80
897/897 [==============================] - 45s 50ms/step - loss: 1.3329 - acc: 0.4991 - val_loss: 1.4547 - val_acc: 0.4776
Epoch 5/80
897/897 [==============================] - 45s 50ms/step - loss: 1.2868 - acc: 0.5159 - val_loss: 1.2856 - val_acc: 0.5391
Epoch 6/80
897/897 [==============================] - 45s 50ms/step - loss: 1.2537 - acc: 0.5290 - val_loss: 1.2529 - val_acc: 0.5330
Epoch 7/80
897/897 [==============================] - 45s 50ms/step - loss: 1.2246 - acc: 0.5413 - v

In [0]:
from keras.models import load_model
classifier = load_model(path + '/emotion_off_cau3.hdf5')

In [0]:
score = classifier.evaluate(X_test, Y_test)
score[1]